 **DM Cours 5 - Pandas**

Dans ce dataset: https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv, chaque ligne correspond à un produit alimentaire mis en vente par un utilisateur.
Objectif: cleaner le dataset.
On aimerait avoir une colonne de prix unifiés en euros. Problème: la currency n'est pas indiquée pour tous les produits: il va falloir essayer de "deviner" les currency manquantes, en se basant sur l'adresse IP de l'utilisateur.
La colonne "infos" liste des ingrédients présents dans le produit. On préfèrerait avoir une colonne de type bool par ingrédient, indiquant si le produit contient ou non cet ingrédient.
Voic une liste d'APIs qui peut vous être utile : https://github.com/public-apis/public-apis (mais vous pouvez en utiliser d'autres si vous le voulez).

In [1]:
import pandas as pd
from requests import get

In [2]:
# import du csv original
products = pd.read_csv('D:\COURS_TELECOM_PARIS\products.csv', sep = ';', na_values='nope')
products.head(5)

,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,NaN,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"


**Utilisation de l´adresse IP pour connaître le pays du consommateur**

In [3]:
# utilisation de l´API ipapi https://ipapi.co/api/?python#introduction
# Exemple d´utilisation
loc = get('https://ipapi.co/58.90.204.239/country_name/')
print(loc.text)
pays=get('https://restcountries.eu/rest/v2/name/France')
print(pays.text)
pays2 = pays.text

SSLError: HTTPSConnectionPool(host='ipapi.co', port=443): Max retries exceeded with url: /58.90.204.239/country_name/ (Caused by SSLError(SSLCertVerificationError("hostname 'ipapi.co' doesn't match either of 'wireless.wifirst.net', 'www.wireless.wifirst.net'")))

In [ ]:
type(pays)

In [ ]:
products = products.dropna(subset = ['ip_address'])

**1.Utilisation de la fonction apply**

In [ ]:
def get_country(df):
    df['api_argument'] = 'https://ipapi.co/' + products['ip_address'].str + '/country_name/'
    loc = get('https://ipapi.co/187.129.113.105/country_name/')
    loc = get(api_argument)

In [ ]:
products['api_argument'] = 'https://ipapi.co/' + products['ip_address'] + '/country_name/'

In [ ]:
s = products['api_argument'].apply(lambda x : get(x).text)
type(s)
s

In [ ]:
products['country'] = s

In [ ]:
del products['api_argument']
products.head(5)

In [ ]:
booleen = products['country'] != 'Undefined'
booleen
products = products[booleen]
products.head(5)

In [ ]:
s = products['country']
s
len(s)
#len(products)

**Currencies: Récupération des codes correspondant aux devises**\
i) Récupération des devises de chaque pays trouvé ci-dessus\
ii) Récupération des codes qui correspondent à ces devises

**RestCountry API**

In [ ]:
from restcountries import RestCountryApiV2 as rapi
import restcountries

In [ ]:
a = rapi.get_countries_by_name('Mexico')
print(((a[0].currencies)[0]['code']))
print(a[0])

In [ ]:
L={}
L[a[0]] = ((a[0].currencies)[0]['code'])
a[0]

In [ ]:
def f(x): return rapi.get_countries_by_name(x)[0].currencies[0]['code']

In [ ]:
# Suppression de l aCorée du Sud, non reconnue par l´API
products = products[products['country']!='South Korea']
s = products['country']
products.head(5)

In [ ]:
def temp(s):
    # dico = {}
    L = []
    for pays_in_s in s:
        #print(pays_in_s)
        try:
            pays = rapi.get_countries_by_name(pays_in_s)
            #print( ((pays[0].currencies)[0])['code'])
            #L[pays_in_s] = ((pays[0].currencies)[0])['code']
            L.append(((pays[0].currencies)[0])['code'])
        except requests.exceptions.InvalidURL:
            L.append('NaN')
    return L

currencies_list = temp(s)

In [ ]:
print(currencies_list)

In [ ]:
len(currencies_list)

products['currency_code'] = currencies_list

**Conversion en euros**

In [ ]:
# Use of the exchange rate API
# https://www.exchangerate-api.com/docs/python-currency-api

In [ ]:
import requests

In [ ]:
url = 'https://api.exchangerate-api.com/v4/latest/EUR'

In [ ]:
response = requests.get(url)
data = response.json()
print(data['rates'])
type(data)

In [ ]:
def rate(s):
    L = []
    # parcours des codes des devises
    for rate in s:
        #print(rate)
        try:
            taux = data['rates'][rate]
            #print(taux)
            L.append(taux)
        except KeyError: # clé non reconnue pour 'NGN'
            L.append('NaN')
    return L

L = rate(products['currency_code'])

products['taux_conversion'] = L  

In [ ]:
products_copie = products.copy()
products.head(5)

**Prix en euros**

In [ ]:
# some cells alredy contain the currency (USD and EUR)
products['price_corrected'] = products['price'][products['price'].str.contains(' ')].str.split(' ').str[0]
products['price_corrected'][products['price'].str.contains(' ') == False] = products['price'][products['price'].str.contains(' ') == False] 

products_copie2 = products.copy()
products

In [ ]:
# conversion en numeric
#products_copie2[products_copie2['taux_conversion']!='NaN']
products = products[products['taux_conversion']!='NaN']

In [ ]:


products['price_corrected'].str.replace(',', ',')
products['price_corrected'].astype(float)

products['taux_conversion'].str.replace('.', ',')
products['taux_conversion'].astype(float)

products.price_corrected = pd.to_numeric(products['price_corrected'], errors = 'coerce')
products.taux_conversion = pd.to_numeric(products['taux_conversion'], errors = 'coerce')

products.head(5)

products['taux_conversion'] * products['price_corrected']

In [ ]:
products.info()

In [ ]:
#products['taux_conversion'] * products['price_corrected']

In [ ]:
products['prix en euros'] = products['price_corrected'] 

In [ ]:
products.head(5)

La colonne "infos" liste des ingrédients présents dans le produit. On préfèrerait avoir une colonne de type bool par ingrédient, indiquant si le produit contient ou non cet ingrédient.

In [ ]:
# List the ingredients contained in the products
def get_ingredients():
    # returns the list of the unique ingredients contained in the table
    L_bad_words=['contain', 'contains', 'Contain', 'Contains',':', ';', ',', 'and', 'May', 'may','or', 'Ingredient', 'Ingredients', 'ingredient', 'ingredients']
    L = []
    for ligne in products['infos']:
        ligne = ligne.replace(':', '').replace(';', '').replace(',', '').split()
        for e in ligne:
            if not (e in L_bad_words):
                if not e in L:
                    print(e)
                    L.append(e.lower())
    return L
get_ingredients()   

In [ ]:

def fill_ingredients():
    # 
    ingredients = get_ingredients() 
    L_bad_words=['contain', 'contains', 'Contain', 'Contains',':', ';', ',', 'and', 'May', 'may','or', 'Ingredient', 'Ingredients', 'ingredient', 'ingredients']
    
    # Création des colonnes par ingrédient et remplissage par défaut à False
    for ingredient in ingredients:
        products[ingredient] = 'False'
    
    i = 0
    for ligne in products['infos']:
        ligne = ligne.replace(':', '').replace(';', '').replace(',', '').split()
        for e in ligne:
            if e.lower() in ingredients:
                products[e.lower()].iloc[i] = True
        i = i + 1            
    return L
 
    
fill_ingredients()

In [ ]:
products['infos'].iloc[0]

In [ ]:
products.head